# The Parity Problem
You are given a function as a black box: $ f: \{0, 1\}^n \rightarrow \{0, 1\}$ and $f(x) = u.x \mod2$.

For some hidden $u \in \{0, 1\}^n$, find $u$ with as few queries as possible. This is a restricted version of the Deutsch–Jozsa algorithm where instead of distinguishing between two different classes of functions, it tries to learn a string encoded in a function. 

## A classical solution

![Classical Solution](bv.PNG "A classical solution the parity problem")

We need $n$ queries to compute the hidden string, because each time you run this circuit, you get only one bit of information. 
For example: let the hidden string $u = 101 $.

$$ f(100) = 1 $$ 

$$ f(010) = 0 $$

$$ f(001) = 1 $$

One possible solution to implement this classically is to use a classical AND gate as a mask to get each bit of information. 

## The Bernstein-Vazirani Algorithm
In contrast to the classical solution, the quantum solution requires only one query to find the hidden $u$. This algorithm was invented by Ethan Bernstein and Umesh Vazirani in 1992. Your task is to implement this algorithm in Q#. You can read more about the algorithm [here](https://en.wikipedia.org/wiki/Bernstein%E2%80%93Vazirani_algorithm).
![Quantum Solution](bv-q.PNG "A quantum solution the parity problem")

The Bernstein-Vazirani algorithm is described as follows:
1. Initialize n query qubits in the state $|0\rangle$ and one answer qubit in the state $|-\rangle$
2. Apply the Hadamard Transform to an n-qubit query state to get $H|0\rangle^{\otimes n} =  \frac{1}{2^{n/2}} \sum_x |x> $.
3. Apply the oracle $U_f$ which transforms the state to  $\frac{1}{2^{n/2}} \sum_x (-1)^{u.x} |x> $
4. Apply the Hadamard Transform to the query state after the oracle is applied.
5. Measure in the computational basis.

In [8]:
import pennylane as qml

## Task 1
**Summary**

Prepare qubits in a uniform superposition. For this, use [qml.broadcast](https://docs.pennylane.ai/en/stable/code/api/pennylane.broadcast.html), instead of a loop. This allows us to apply a unitary multiple times to a specific pattern of wires.

**Input**

*query_wires*: an array of numbers representing the query wires

Apply the hadamard transform the query_wires. 


In [9]:
def walshHadamardTransform(query_wires):
    qml.broadcast(unitary=qml.Hadamard, wires=query_wires, pattern="single")

## Task 2
**Summary**

Prepare the answer wire (traditionally, the last qubit on our device) in the state $|-\rangle$

**Input**

*answer_wire*: the answer qubit

In [10]:
def prepareAnswerQubit(answer_wire):
    qml.PauliX(wires=answer_wire)
    qml.Hadamard(wires=answer_wire)

## Task 3

**Summary**

Encode the secret string (supplied as a boolean array s) into a function $U_f$

**Input**: 
query_wires: qubit array
answer_wire: answer qubit
secret: secret string as a boolean array e.g. [True, False, True, True]


In [11]:
def Uf(query_wires, answer_wire, secret):
    for i, s in enumerate(secret):
        if s:  # If the secret bit is True/1, apply CNOT with the query wire as control and answer wire as target
            qml.CNOT(wires=[query_wires[i], answer_wire])

## Task 4: Explain what gates you applied to create the encoded string as $U_f$ and why you applied them.


To encode the secret string into the function Uf, I apply Controlled-NOT (CNOT) gates.
A CNOT gate flips the state of the target qubit if and only if the control qubit is in the state |1⟩.
In this case, the control qubits are the query qubits, and the target qubit is the answer qubit.
So, for each bit in the secret string that is True, we apply a CNOT gate controlled by the corresponding query qubit, 
targeting the answer qubit. This effectively encodes the secret string into the function Uf.


## Task 5: Run the Bernstein-Vazirani Algorithm using operations defined above

1. Create the secret string, and the required qubits
1. Perform the Bernstein Vazirani algorithm by using the tasks above. 
1. Return probabilities on the query_wires.


In [12]:
# Change the secret string to encode a string of your choice
secret_string = [True, False, False, True]

query_wires = range(len(secret_string))
answer_wire = len(secret_string)

dev = qml.device('default.qubit', wires=(len(secret_string) + 1), shots=1024) # + 1 for the answer wire

In [13]:
@qml.qnode(dev)
def BernsteinVazirani(query_wires, answer_wire, secret_string):
    query_wires = range(len(secret_string))
    answer_wire = len(secret_string)

    # Step 1: Apply Hadamard transform to all query wires to prepare them in a superposition state
    walshHadamardTransform(query_wires)

    # Step 2: Prepare the answer qubit in the |-> state
    prepareAnswerQubit(answer_wire)

    # Step 3: Apply the oracle function Uf
    Uf(query_wires, answer_wire, secret_string)

    # Step 4: Apply Hadamard again to the query wires
    walshHadamardTransform(query_wires)

    # Step 5: Measure the query wires to find the secret string
    return qml.counts(wires=query_wires)


### Check whether your answer corresponds to the secret string

In [14]:
BernsteinVazirani(query_wires, answer_wire, secret_string)

{'1001': tensor(1024, dtype=int64, requires_grad=True)}